# Approach
We update the decision boundary (a) and the rate of evidence accumulation (v) using estimates of the reward difference between targets ($\Delta B$) and the reward changepoint probability ($\Omega$). 

#### Nomenclature
&emsp;

**Learning signals (estimates from ideal observer)**

&emsp;

${\Delta B} = \textrm{signed belief in the reward difference between optimal \& suboptimal targets or target identities}$ 

$\Omega = \textrm{change point probability}$ 

&emsp;

**Learning targets (decision parameters)**

&emsp;

$a = \textrm{decision boundary}$

$v = \textrm{drift rate}$

&emsp;

**Other parameters involved in calculating the above**

&emsp;

$\sigma^2_n = \textrm{variance of the generative distribution}$

$\sigma^2_t = \textrm{estimated variance}$

$\phi = \textrm{model confidence}$

$H = \textrm{hazard rate}$

$r_{t} = \textrm{reward observed}$

$\alpha = \textrm{bayesian belief learning rate}$

$\delta = \textrm{reward prediction error}$

$RU = \textrm{reward uncertainty}$

&emsp;



## Calculation 

The following algorithm is adapted from [Vaghi et al., 2017](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5643443/)  ([code repo](https://github.com/BDMLab/Vaghi_Luyckx_et_al_2017); [main function used to calculate ideal observer parameters](https://github.com/BDMLab/Vaghi_Luyckx_et_al_2017/blob/master/Functions/redBayes_circ.m)). A detailed description of their model is under Method Details --> Computational Model. 

&emsp;

Note that there is an error in the calculation of model confidence in their manuscript (equation 8). There, model confidence ($\phi$) is defined as the calculation for reward uncertainty (RU) instead of (1 - RU) (but their code is correct, and we use the correct calc. of model confidence). 

&emsp;

(Note that this approximation of a Bayesian delta-rule model was first proposed by [Nassar et al. 2010](https://www.jneurosci.org/content/30/37/12366) and Vaghi et al. based their model on the reduced Bayesian observer in [Mcguire et al. 2014](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4437663/)).

### Belief 
Given that $c$ = the chosen target and $u$ = the unchosen target, the belief in the mean of the distribution of reward differences on the next trial is calculated as:
$$B_{t+1, c} = B_{t, c}  + \alpha_t\delta_t$$

The unchosen target value decays to the pooled expected value of both targets, $E(r)$:
$$B_{t+1, u} = B_{t, u}(1-\Omega_t)+\Omega_tE(r)$$
<br>
$$E(r) = \frac{\bar{r}_{t_0} + \bar{r}_{t_1}}{2}$$

The signed belief in the reward difference between targets is calculated as the difference in belief for targets 0 and 1:
$$\Delta B = B_{t,1} - B_{t,0}$$

For the purpose of visualization and analysis, this can be recast as the reward difference between optimal & suboptimal targets:
$$\Delta B_{t, opt} = B_{t,opt} - B_{t,subopt}$$

The learning rate of the model [$\alpha$] is influenced by the change point probability [$\Omega$] and the model confidence [$\phi$]. The learning rate should be high if either 1) a change in the mean of the distribution of reward is likely [$\Omega$ is high] or 2) the estimate of the mean is highly imprecise [$\sigma^2_n$ is high]:
$$\alpha_t = \Omega_t + (1-\Omega)(1-\phi_t)$$
The prediction error, $\delta$, is the difference between the model belief and the reward difference observed:
$$\delta_t = r_t - B_{t, c}$$
Estimated variance is calculated as:
$$\sigma^2_t = \sigma^2_n + \frac{(1-\phi_t)\sigma^2_n}{\phi_t}$$

### Changepoint probability 
The changepoint probability is the likelihood that a new sample is drawn from the same Gaussian distribution centered about the current belief estimate of the model relative to the likelihood that a new sample is drawn from a uniform distribution. The changepoint probability will be close to 1 as the relative probability of a sample coming from a uniform distribution increases.
$$\Omega_t = \frac{U(r_{\Delta_{t}})H}{U(r_{\Delta_{t}})H + N(r_{\Delta_{t}}| B_{\Delta_{t}},\sigma^2_t)(1-H)}$$
The hazard rate is the global probability that the mean of the distribution has changed (calculated as the sum of change points over the total number of trials).
$$H = \frac{sum(cp_{trials})}{n_{trials}}$$


The model confidence [$\phi$] is a function of the changepoint probability [$\Omega$] and the variance of the generative distribution [$\sigma^2_n$]. The first term is the variance when a changepoint is assumed to have occurred. The second term is the variance conditional on no changepoint (slowly decaying uncertainty). The third term is the rise in uncertainty when the model is unsure whether a changepoint has occurred. The same terms are in the denominator with an added variance term to reflect uncertainty arising from noise.
$$RU_t = \frac{\Omega_t\sigma^2_n + (1-\Omega_t)(1-\phi_t)\sigma^2_n + \Omega_t(1-\Omega_t)(\delta_t\phi_t)^2}{\Omega_t\sigma^2_n + (1-\Omega_t)(1-\phi_t)\sigma^2_n + \Omega_t(1-\Omega_t)(\delta_t\phi_t)^2+\sigma^2_n}$$

$$\phi_{t+1} =  1 - RU$$

We propose that the belief in the relative reward for the two choices, $B$, updates the drift rate, $v$, or the speed of evidence accumulation:
 
$$v_{t+1} = \hat{\beta}_{v}\cdot B_{\Delta_{t}} + v_t$$

and that the change point probability, $\Omega$ decreases the decision threshold, $a$, or the amount of evidence needed to make a decision:  

$$a_{t+1} = a_0 - \hat{\beta}_{a}\cdot \Omega_{t}$$


# Computational process for generating parameters

0) [Generate experimental parameters](https://github.com/kmbond/loki_1/blob/ff8448c925241d26698ea2d15ce40600e718c8d9/generative_scripts/genTrialStructure_greeblified_decisions.ipynb
) for each subject, varying the frequency of change points and the conflict between choices on a given trial.

1) The [ideal observer calculation routine](
https://github.com/kmbond/loki_1/blob/master/analysis/pseudobayes/bayesian_belief_model_p_ext.py) receives the same experimental parameters as given to the subjects. For each subject and for each session, extract the reward values and change points for each target from the experimental parameters. 
* `expParam = pd.read_csv(experimental_parameters)`
* `p_targets = expParam.reward_p_t0, expParam.reward_p_t1` 

2) Calculate derivatives to feed to the ideal observer calculation function.

* `reward_difference = expParam.reward_p_t0 - expParam.reward_p_t1`
* `H = expParam[(expParam.cp == 1)].shape[0] / expParam.shape[0] # hazard rate`
* `low = p_targets.min() # min reward value`
* `up = p_targets.max() # max reward value` 
* `high = up - low # reward value range` 


3) [Update the Bayesian parameters](https://github.com/kmbond/loki_1/blob/master/analysis/pseudobayes/simulation_functions_loki.py) for each trial within a session. 

Inputs: Trial number (t), total number of trials (nTrials), reward values for each target (prob_reward_targets), the hazard rate (H), true variance of the generative reward distribution (sN), low (reward value min), up (reward value max), high (reward value range), choices (array of choices so far), B (belief in the value of each target), signed_B_diff (belief in target 1 - belief in target 0), B_diff (belief in chosen target - belief in unchosen target), lr (learning rate), rpe (reward prediction error), cpp (change point probability), MC (model confidence), epoch_length, sF (estimated variance of the generative reward distribution)).

Outputs: Updated B, signed_B_diff, B_diff, lr, rpe, CPP, MC, epoch_length, sF 

`B,signed_B_diff,B_diff,lr,rpe,CPP,MC,epoch_length,sF = update_bayesian_belief(t, nTrials, prob_reward_targets, H, sN, low, up, high, choices, B, signed_B_diff, B_diff, lr, rpe, CPP, MC, epoch_length, sF)`

These updated outputs are then given as inputs for the next trial.


4) [Repeat this procedure](https://github.com/kmbond/loki_1/blob/master/analysis/pseudobayes/binary_ddm_rl_simulation_p.py) for all subjects  and all sessions using multiprocessing. 

All output for each session is [saved as a pickled object](https://github.com/kmbond/loki_1/tree/master/data/simulated_data
) under `data/simulated_data/`.

e.g. `sim789_reward6.pkl` would refer to the ideal observer parameters calculated for subject 789 for the session with [reward code 6](https://github.com/kmbond/loki_1/blob/master/reward_condition_key.csv) (reward probability of 85% for the optimal choice and a switch in reward contingencies every 10 trials on average).

5) Select output contained within these pickled objects is [saved for future analysis](https://github.com/kmbond/loki_1/blob/master/analysis/pseudobayes/pickle_objects.py) within a csv file (e.g. `sub-789_cond-8510_learning_signals.csv`).


--


6) If also simulating predicted changes in the drift-rate or the decision boundary as a function of the learning signals $\Delta B$ and $\Omega$, then the drift-rate and decision boundary are updated according to the model and the learning rate specified in the [multiprocessing script](https://github.com/kmbond/loki_0.5/blob/master/analysis/pseudobayes/binary_ddm_rl_simulation_p.py) and the [simulation class](https://github.com/kmbond/loki_0.5/blob/master/analysis/pseudobayes/functions/simulation_functions_loki.py). Trial-by-trial boundary and drift-rate estimates are saved as part of the .pkl object for each session. Note that I have only included the specification for a single model for the sake of brevity / clarity. 
